In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import json
import numpy as np
import os
import random
import scipy.io as sio
import torch
import torch.nn as nn
#from torchsummary import summary
STEP =256

In [2]:
#Padded input data , labels are arrays which are also padded to maximum size , 
#its not one hot encoded. 
data = np.load('train.npz')
train_x = data['a']
train_y = data['b']
print(train_x.shape,train_y.shape)

(7676, 18176, 1) (7676, 71)


In [3]:
#just an generator for training.
def data_generator(batch_size, x, y):
    num_examples = len(x)
    examples = zip(x, y)
    examples = sorted(examples, key = lambda x: x[0].shape[0])
    end = num_examples - batch_size + 1
    batches = [examples[i:i+batch_size]
                for i in range(0, end, batch_size)]
    random.shuffle(batches)
    batchlen = int(len(batches))
  
    i =0
    while True:

        if i >= batchlen:
               return
        #print(i)
        x, y = zip(*batches[i])
        yield np.asarray(x), np.asarray(y)
        i = i +1
        
            #yield x,y

In [4]:
class Bn_Relu(nn.Module):
    def __init__(self, output,dropout=0):
        super(Bn_Relu,self).__init__()
        self.bn =  nn.BatchNorm1d(output)
        self.relu = nn.ReLU()
        self.drop = None
        if dropout > 0:
           self.drop = nn.Dropout(dropout)
    def forward(self,x):
        out = self.bn(x)
        out = self.relu(out)
        if self.drop is not None:
            out = self.drop(out)
        return out

In [5]:
class ResnetBlock(nn.Module):
    def __init__(self,prev_filters,num_filters,subsample_length,block_index,dropout):
        super(ResnetBlock, self).__init__()
        self.subsample_length = subsample_length
        self.max1 = nn.MaxPool1d(kernel_size=subsample_length)
        self.zero_pad = (block_index % 4) == 0 and block_index > 0
        #if zero_pad is True:
            #shortcut = Lambda(zeropad, output_shape=zeropad_output_shape)(shortcut)
        print("Conv 1" ,prev_filters,num_filters,subsample_length)
        self.conv1 = nn.Conv1d(prev_filters, num_filters, 3, subsample_length,padding=1)
        self.bn1 = None
        if(block_index !=0):
              self.bn1 = Bn_Relu(num_filters, dropout )
        print("Conv 2" ,num_filters)
        self.downsample = None
        if( subsample_length != 1 ) or (prev_filters != num_filters):
            self.downsample = nn.Sequential(
                nn.Conv1d(prev_filters, num_filters, 3, subsample_length,padding=1),
                nn.BatchNorm1d(num_filters))
            
        self.conv2 = nn.Conv1d(num_filters, num_filters,3, 1,1)
    
    def forward(self,x):
        iden = x #self.max1(x)
                       
        #print ("Shape After max ",iden.shape,self.subsample_length)    
        out = self.conv1(x)
        if self.bn1 is not None:
            out = self.bn1(out)
        out = self.conv2(out)
        if self.downsample is not None:
            iden = self.downsample(iden)
        #print ("out Before ",out.shape,iden.shape)
        
        
        #print ("Shape first ,second",iden.shape, out.shape)
        out += iden
        return out       

In [6]:
def get_num_filters_at_index(index, num_start_filters):
    return 2**int(index / 4) \
        * num_start_filters

In [7]:
class ResNet(nn.Module):

    def __init__(self,dropout):
        super(ResNet, self).__init__()
        self.layers1 = nn.Conv1d(1, 32, 16, 1)
        self.bn1 = Bn_Relu(32)
        layers = []
        res = [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
        #res = [1,2,1]
        prev_filter =32
        self.debug =0;
        for index, subsample_length in enumerate(res):
            num_filters = get_num_filters_at_index(index, 32 )
            print("filter length",prev_filter,num_filters)
            x =  ResnetBlock(prev_filter,num_filters,subsample_length,index,dropout)
            y = Bn_Relu(num_filters,dropout)
            prev_filter = num_filters
            layers.append(x)
            layers.append(y)
        self.resLayer = nn.Sequential(*layers)
        self.deconv = nn.ConvTranspose1d(256,256,2)
        self.linear1 = nn.Linear(256,4)
        
    def forward(self, x):
        #print(self.debug)
        self.debug +=1
        x= self.layers1(x)
        x= self.bn1(x)
        x = self.resLayer(x)
        #print("shape before",x.shape)
        #x = F.pad(x, (0,1), "constant", 0) 
        #print("shape before",x.shape)
        #x = self.deconv(x)
        x = x.permute(0,2,1)
        
        x = self.linear1(x)
        #y = x.view(x.size()[0], -1);
        #print ("final shape ", x.shape)
        #return F.log_softmax(self.linear1(x.view(x.size()[0], -1)), dim=1)
        return F.log_softmax(x, dim=1)
        #return x
        
        

In [8]:
import torch.nn.functional as F
loss_fn = nn.CrossEntropyLoss(size_average=False)

C:\anaconda3\envs\fresh\lib\site-packages\torch\nn\_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [9]:
model = ResNet(dropout =0.2)

filter length 32 32
Conv 1 32 32 1
Conv 2 32
filter length 32 32
Conv 1 32 32 2
Conv 2 32
filter length 32 32
Conv 1 32 32 1
Conv 2 32
filter length 32 32
Conv 1 32 32 2
Conv 2 32
filter length 32 64
Conv 1 32 64 1
Conv 2 64
filter length 64 64
Conv 1 64 64 2
Conv 2 64
filter length 64 64
Conv 1 64 64 1
Conv 2 64
filter length 64 64
Conv 1 64 64 2
Conv 2 64
filter length 64 128
Conv 1 64 128 1
Conv 2 128
filter length 128 128
Conv 1 128 128 2
Conv 2 128
filter length 128 128
Conv 1 128 128 1
Conv 2 128
filter length 128 128
Conv 1 128 128 2
Conv 2 128
filter length 128 256
Conv 1 128 256 1
Conv 2 256
filter length 256 256
Conv 1 256 256 2
Conv 2 256
filter length 256 256
Conv 1 256 256 1
Conv 2 256
filter length 256 256
Conv 1 256 256 2
Conv 2 256


In [10]:
model.cuda()

ResNet(
  (layers1): Conv1d(1, 32, kernel_size=(16,), stride=(1,))
  (bn1): Bn_Relu(
    (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (resLayer): Sequential(
    (0): ResnetBlock(
      (max1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (1): Bn_Relu(
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (drop): Dropout(p=0.2)
    )
    (2): ResnetBlock(
      (max1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
      (bn1): Bn_Relu(
        (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (drop): Dr

In [11]:
import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
def TrainModel(epoch,batchsize):
    model.train()    
    for epoch in range(epoch):
        print('*'*10)
        print('Epoch: {}'.format(epoch))
        tloss = 0.0
        taccu = 0.0
        train_gen = data_generator(batchsize,train_x,train_y)
        for x,y in train_gen:
            #print (x.shape,y.shape)
            inp = np.transpose(x, (0, 2, 1))
            inp =torch.from_numpy(inp).requires_grad_()
            inpTensor= inp.cuda()
            optimizer.zero_grad()
            outputs = model(inpTensor)
            labels = torch.from_numpy(y).long()
            labels = labels.cuda()
            loss = loss_fn(outputs.permute(0,2,1),labels)
            #loss = loss_fn(outputs.view(outputs.shape[0]*outputs.shape[1],-1), labels.view(labels.shape[0]*labels.shape[1]))
            tloss += loss.item()
            pred = torch.max(outputs, -1)[1]
            taccu += (pred == labels).sum().item()
            #print("loss ", loss)
            loss.backward()
            optimizer.step()
            #break
            
        print('Loss: {}'.format(tloss))
        print('Accu: {}'.format(taccu/len(train_x)))

In [13]:
def ValidateModel(batchsize):
    model.eval()
    tloss = 0.0
    taccu = 0.0
    valid_gen = data_generator(batchsize,valid_x,valid_y)
    for x,y in valid_gen:
        inp = np.transpose(x, (0, 2, 1))
        inp =torch.from_numpy(inp)
        inpTensor= inp.cuda()
        outputs = model(inpTensor)
        labels = torch.from_numpy(y).long()
        labels = labels.cuda()
        loss = loss_fn(outputs.permute(0,2,1),labels)
        #loss = loss_fn(outputs.view(outputs.shape[0]*outputs.shape[1],-1), labels.view(labels.shape[0]*labels.shape[1]))
        tloss += loss.item()
        pred = torch.max(outputs, -1)[1]
        taccu += (pred == labels).sum().item()
    print('Loss: {}'.format(tloss))
    print('Accu: {}'.format(taccu/len(valid_x)))

In [14]:
data = np.load('valid.npz')
valid_x = data['a']
valid_y = data['b']
print(valid_x.shape,valid_y.shape)

(852, 18176, 1) (852, 71)


In [15]:
if(os.path.exists("model-latest.pth")) :
	model.load_state_dict(torch.load("model-latest.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [17]:
torch.cuda.empty_cache()
TrainModel(1,32)

**********
Epoch: 0
Loss: 69546.4031829834
Accu: 66.92339760291819


In [17]:
TrainModel(1,32)

**********
Epoch: 0
Loss: 262899.7056274414
Accu: 55.71430432516936


In [18]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 22425.94314956665
Accu: 63.44600938967136


In [19]:
torch.save(model.state_dict(), "test-1.pth")

In [20]:
torch.cuda.empty_cache()
TrainModel(5,32)

**********
Epoch: 0
Loss: 233093.6732788086
Accu: 58.023579989577904
**********
Epoch: 1
Loss: 208672.001953125
Accu: 59.460005211047424
**********
Epoch: 2
Loss: 199116.2645263672
Accu: 59.92509119332986
**********
Epoch: 3
Loss: 199087.25582885742
Accu: 59.963262115685254
**********
Epoch: 4
Loss: 193717.21255493164
Accu: 60.28947368421053


In [22]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 22302.803985595703
Accu: 60.063380281690144


In [21]:
torch.save(model.state_dict(), "test-1.pth")

In [23]:
torch.cuda.empty_cache()
TrainModel(6,32)

**********
Epoch: 0
Loss: 153722.51000976562
Accu: 63.23150078165711
**********
Epoch: 1
Loss: 151669.26217651367
Accu: 63.30875455966649
**********
Epoch: 2
Loss: 150966.83184814453
Accu: 63.22146951537259
**********
Epoch: 3
Loss: 148333.6430053711
Accu: 63.451146430432516
**********
Epoch: 4
Loss: 146318.73986816406
Accu: 63.577384054194894
**********
Epoch: 5
Loss: 143104.78533935547
Accu: 63.5872850442939


In [24]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 21864.313079833984
Accu: 60.225352112676056


In [41]:
torch.save(model.state_dict(), "test-1.pth")
model.load_state_dict(torch.load("test-1.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [39]:
torch.cuda.empty_cache()
TrainModel(6,32)

**********
Epoch: 0
Loss: 142182.44482421875
Accu: 63.65046899426785
**********
Epoch: 1
Loss: 140148.30081176758
Accu: 63.661021365294424
**********
Epoch: 2
Loss: 141206.31518554688
Accu: 63.642391870766026
**********
Epoch: 3
Loss: 141265.24829101562
Accu: 63.68981240229286
**********
Epoch: 4
Loss: 137422.86274719238
Accu: 63.84275664408546
**********
Epoch: 5
Loss: 137796.7368774414
Accu: 63.823736321000524


In [40]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 18555.244667053223
Accu: 62.570422535211264


In [28]:
torch.save(model.state_dict(), "test-2.pth")

In [29]:
model.load_state_dict(torch.load("test-2.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [30]:
torch.cuda.empty_cache()
TrainModel(6,16)

**********
Epoch: 0
Loss: 136388.07900238037
Accu: 63.86099531005732
**********
Epoch: 1
Loss: 132161.95597076416
Accu: 64.10265763418447
**********
Epoch: 2
Loss: 131530.73462677002
Accu: 64.1323606044815
**********
Epoch: 3
Loss: 129756.62490463257
Accu: 64.34236581552892
**********
Epoch: 4
Loss: 126288.5089263916
Accu: 64.35330901511203
**********
Epoch: 5
Loss: 126232.15660095215
Accu: 64.3882230328296


In [31]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 15539.197761535645
Accu: 63.617370892018776


In [32]:
torch.save(model.state_dict(), "test-2.pth")